In [690]:
import pandas as pd
import json

Per la realizzazione ho seguito https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

In [691]:
dataset = pd.read_csv('description_predictions.csv', index_col=0)
with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [692]:
dataset_withoutNAN = dataset[dataset.prediction != 'NAN']

dataset_withoutNAN.reset_index(drop=True, inplace=True)

Il codice seguente serve per convertire i soprannomi delle squadre in home team e away team

In [693]:
for i, row in dataset_withoutNAN.iterrows():
    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction

    syn = {}
    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match
    for key in jsonObject.keys():
        if (h_team in key) or (key in h_team):
            syn['home team'] = jsonObject[key] 
            
        if (a_team in key) or (key in a_team):
            syn['away team'] = jsonObject[key] 

    #successivamente prendo il testo e sostituisco i sinonimi con home o away team
    description = description.lower()
    for key in syn.keys():
        for val in syn[key]:
            description = description.replace(val.lower(), key)

    dataset_withoutNAN.at[i, 'description'] = description

In [694]:
dataset_withoutNAN = dataset_withoutNAN[['description', 'prediction']]
prediction_labels = {
    'N': 0,
    'V': 1,
    'P': 2
}

dataset_withoutNAN['prediction'] = dataset_withoutNAN['prediction'].map(prediction_labels)
dataset_withoutNAN.head()

,description,prediction
0,home team and away team take on each other in ...,0
1,home team begin their title defence with a mat...,1
2,away team will look to make a flying start to ...,2
3,newcomers home team will be aiming to kick off...,2
4,after failing to impress in the 2020/21 serie ...,2


In [695]:
from nltk.corpus import stopwords
import nltk
stop_words = set(stopwords.words('english'))

# this function returns a list of tokenized and stemmed words of any text
def get_tokenized_list(doc_text):
    tokens = nltk.word_tokenize(doc_text)
    return tokens

# This function will performing stemming on tokenized words
def word_stemmer(token_list):
  ps = nltk.stem.PorterStemmer()
  stemmed = []
  for words in token_list:
    stemmed.append(ps.stem(words))
  return stemmed

In [696]:
# Function to remove stopwords from tokenized word list
def remove_stopwords(doc_text):
  cleaned_text = []
  for words in doc_text:
    if words not in stop_words:
      cleaned_text.append(words)
  return cleaned_text

In [697]:
cleaned_corpus = []
for doc in dataset_withoutNAN.description:
  tokens = get_tokenized_list(doc)
  doc_text = remove_stopwords(tokens)
  doc_text  = word_stemmer(doc_text)
  doc_text = ' '.join(doc_text)
  cleaned_corpus.append(doc_text)

In [698]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [699]:

vectorizer = TfidfVectorizer(ngram_range=(2,2)) #vectorizer sarà il nostro modello da allenare
text_counts = vectorizer.fit_transform(cleaned_corpus) #impara il vocabolario e crea Idf, poi ritorna la matrice document-term
df1 = pd.DataFrame(text_counts.toarray(), columns=vectorizer.get_feature_names())

In [700]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, dataset_withoutNAN.prediction, test_size=0.2, random_state=1)

In [701]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3).fit(X_train)


Nel seguente codice applico il k-means

In [702]:
import sklearn.metrics as mtr
import numpy as np 

label_classes = {
    'N': 0,
    'V': 1,
    'P': 2
}

classes = [0,1,2]

import itertools

#trovo tutte le combinazioni da 3 classi (0 = 'N', 1 = 'V', 2 = 'P')
combinations = list(itertools.product(*[classes,classes,classes]))
combs = {}
i = 0

#filtro le combinazioni con le 3 classi diverse: la funzione precedente mi dà anche AAA AAC AAB ecc. sono classi sporche, quindi le elimino
for comb in combinations:
    dict_comb={}
    if comb[0]!=comb[1] and comb[0]!=comb[2] and comb[1]!=comb[2]:
        dict_comb['N']=comb[0]
        dict_comb['V']=comb[1]
        dict_comb['P']=comb[2]
        combs[i]=dict_comb
        i+=1

Abbiamo ottenuto diverse combinazioni di classi, ma avendo le 3 classi in ciascuna combinazione, non è detto che le etichette corrispondano al cluster corretto. 
Per questo motivo vado a trovare i cluster che mi danno un maggior score. 
Per questo ho creato un dizionario con associata la classe: 
- per ciascuna combinazione delle classi vado a sostituire il valore con un carattere temporaneo
- sostituisco i valori dei caratteri con le relative classi della combinazione

Faccio questo perché se avessi una predizione [0, 1, 0, 2], magari le etichette corrette hanno [1, 0, 1, 2] quindi le classi sono corrette ma sono sbagliati i nomi. Quindi devo cambiare i nomi delle varie classi e prendere la combinazione che mi dà il maggior valore

In [707]:
best_score = 0 #nessun kmeans darà 0, quindi se lo trovo maggiore ho trovato il miglior score
best_kmeans = combs[0]

prediction = kmeans.predict(X_test)
original_prediction = prediction.copy()
for combination, dict_classes in combs.items():
    if combination != 0: #la prima combinazione è già fatta, quindi la salto
        prediction = original_prediction.copy()
        #prima cambio il contenuto dell'array con numeri temporanei
        for letter, single_class in dict_classes.items():
            prediction[prediction == single_class] = ord(letter) #ord permette di ottenere il codice decimale del carattere

        #sostituisco i relativi valori con i relativi valori iniziali 
        for letter, single_class in label_classes.items():
            prediction[prediction == ord('N')] = 0
        
    score = mtr.accuracy_score(y_test, prediction)
    print(prediction, score)
    if score > best_score:
        best_score = score
        best_kmeans = dict_classes #tengo la combinazione delle classi

[1 1 2 1 1 2 2 2 1 1 2 2 1 1 1 2 2 1 1 2 1 1 2 1 1 1 1 0 1 1 1 1 1 1 2 2 2
 2 1 2 1 1 1 2 0 2 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 2 1 2 1 1 0 1 0 0 2 1
 1 0] 0.4342105263157895
N 0
V 2
P 1
[2 2 1 2 2 1 1 1 2 2 1 1 2 2 2 1 1 2 2 1 2 2 1 2 2 2 2 0 2 2 2 2 2 2 1 1 1
 1 2 1 2 2 2 1 0 1 2 2 2 2 2 2 2 0 2 2 2 0 2 2 2 2 2 1 2 1 2 2 0 2 0 0 1 2
 2 0] 0.35526315789473684
N 1
V 0
P 2
[0 0 2 0 0 2 2 2 0 0 2 2 0 0 0 2 2 0 0 2 0 0 2 0 0 0 0 1 0 0 0 0 0 0 2 2 2
 2 0 2 0 0 0 2 1 2 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 2 0 2 0 0 1 0 1 1 2 0
 0 1] 0.2894736842105263
N 1
V 2
P 0
[0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 2 0 0 0 0 0 0 1 1 1
 1 0 1 0 0 0 1 2 1 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 1 0 1 0 0 2 0 2 2 1 0
 0 2] 0.2894736842105263
N 2
V 0
P 1
[2 2 0 2 2 0 0 0 2 2 0 0 2 2 2 0 0 2 2 0 2 2 0 2 2 2 2 1 2 2 2 2 2 2 0 0 0
 0 2 0 2 2 2 0 1 0 2 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 0 2 0 2 2 1 2 1 1 0 2
 2 1] 0.27631578947368424
N 2
V 1
P 0
[1 1 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 1 1 1 2 1 1 1 

Converto le classi secondo il dizionario con il massimo score. 
Non posso convertire ad esempio 0 in 1, 1 in 2 e 2 in 0, perché alla fine avrò tutti i numeri uguali, quindi devo usare un passaggio intermedio utilizzando altri numeri.

In [704]:

for letter, single_class in best_kmeans.items():
    original_prediction[original_prediction == single_class] = ord(letter)

for letter, single_class in best_kmeans.items():
    original_prediction[original_prediction == ord(letter)] = single_class
    
score = mtr.accuracy_score(y_test, original_prediction)
print(original_prediction, score)

[1 1 2 1 1 2 2 2 1 1 2 2 1 1 1 2 2 1 1 2 1 1 2 1 1 1 1 0 1 1 1 1 1 1 2 2 2
 2 1 2 1 1 1 2 0 2 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 2 1 2 1 1 0 1 0 0 2 1
 1 0] 0.4342105263157895
